In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D,UpSampling1D
from keras.models import Model
from keras import backend as K

n_features = 4
boat_csv = pd.read_csv("Data/Boat_nominal_data/Boat_sequences_mix.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0", "M0C", "M1C", 
                                  "Acceleration","Speed"])
scaler = StandardScaler()
normal_data = scaler.fit_transform(boat_csv)
print(normal_data.shape)

boat_val = pd.read_csv("Data/Boat_nominal_data/Boat_sequence_mix_val.csv")
boat_val = boat_val.drop(columns=["Unnamed: 0", "M0C", "M1C", 
                                  "Acceleration","Speed"])
scaler = MinMaxScaler(feature_range=(0,1))
val_nom_data = scaler.fit_transform(boat_val)

def prepare_sequences(data, batch_size):
    samples = []
    for i in range(0,data.shape[0], batch_size):
        sample = data[i:i+batch_size]	
        samples.append(sample)
    sequences = np.array(samples)
    trainX = np.reshape(sequences, (len(sequences), batch_size, n_features))
    return trainX


def prepare_data():    
    trainX_nominal = prepare_sequences(normal_data,1024) 
    print(trainX_nominal.shape)
    
    valX_nominal = prepare_sequences(val_nom_data,1024)
    print(valX_nominal.shape)

    return trainX_nominal, valX_nominal

trainX_nominal, valX_nominal = prepare_data()


Using TensorFlow backend.


(307200, 4)


(300, 1024, 4)
(30, 1024, 4)


In [53]:
from keras import Sequential
from keras.layers import MaxPooling1D, MaxPool1D, Conv2DTranspose, BatchNormalization, RepeatVector
import tensorflow as tf

input_shape = (None, n_features)
latent_dim = 20
kernel_size = 2
use_mse = True   
load_weights = False
random_weight = 1


def create_vae(beta):
    filters = 16
    units = len(trainX_nominal)
    units = 30

    def sampling(args):
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        # by default, random_normal has mean=0 and std=1.0
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

    
    def Conv1DTranspose(input_tensor, filters, kernel_size,last, strides=2, padding='same'):
        if last:
            activation = 'linear'
        else:
            activation = 'relu'
        x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
        x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), 
                            strides=(strides, 1), padding=padding,
                            activation=activation)(x)
        x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
        return x


    inputs = Input(shape=input_shape, name='encoder_input')
    x = inputs
    for i in range(5):
        #filters *= 2
        # x = Conv1D(filters=filters,
        #            kernel_size=kernel_size,
        #            padding='same',
        #            activation='relu',
        #            strides=2)(x)
        x = Dense(units=units, activation='relu')(x)
    #shape = K.int_shape(x)

    
    x = Flatten()(x)
    x = Dense(30, activation='relu')(x) 
    #x = BatchNormalization(momentum=0.0)(x)
    z_mean = Dense(latent_dim, name='z_mean')(x)
    #z_mean = BatchNormalization(momentum=0.0)(z_mean)
    z_log_var = Dense(latent_dim, name='z_log_var')(x)
    #z_log_var = BatchNormalization(momentum=0.0)(z_log_var)
    z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
    
    encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
    encoder.summary()

    latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
    x = Dense(units=units, name='Dense_after_sampling', activation='relu')(latent_inputs)

    filters = 50
    for i in range(4):
        # x = Conv1DTranspose(input_tensor=x,
        #                     filters=filters, 
        #                     kernel_size=kernel_size, 
        #                     padding='same',
        #                     last=False)
        #filters //= 2
        #x = UpSampling1D(size=2)(x)
        x = Dense(units=units, activation='relu')(x)

    
    # outputs = Conv1DTranspose(input_tensor=x,
    #                           filters=n_features, 
    #                           kernel_size=2, padding='same',
    #                           last=True)
    outputs = RepeatVector(1024)(x)
    outputs = Dense(n_features)(outputs)

    decoder = Model(latent_inputs, outputs, name='decoder')
    decoder.summary()

    outputs = decoder(encoder(inputs)[2])
    vae = Model(inputs, outputs, name='vae')
    from keras.losses import mse

    reconstruction_loss = 1024*mse(K.flatten(inputs), K.flatten(outputs))
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    loss = K.mean(reconstruction_loss)
    
    vae.add_loss(loss)

    vae.compile(optimizer='rmsprop')

    return (vae, encoder, decoder)


from keras.callbacks import ModelCheckpoint
betas = np.linspace(1,1,1)
print(betas)


vaes = []
for i, beta in enumerate(betas):
    print("Creating VAE with beta=", betas[i])
    vaes.append(create_vae(beta))

[1.]
Creating VAE with beta= 1.0


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 1024, 4)      0                                            
__________________________________________________________________________________________________
dense_123 (Dense)               (None, 1024, 30)     150         encoder_input[0][0]              
__________________________________________________________________________________________________
dense_124 (Dense)               (None, 1024, 30)     930         dense_123[0][0]                  
__________________________________________________________________________________________________
dense_125 (Dense)               (None, 1024, 30)     930         dense_124[0][0]                  
__________________________________________________________________________________________________
dense_126 

In [54]:

for i in range(len(vaes)):
    # checkpointer = ModelCheckpoint(filepath="Models/Weights/Nominal_weights_vae.hdf5",
    #                                verbose=1, save_best_only=True)
    print("FITTING Vae with beta =", betas[i])
    vaes[i][0].fit(x=trainX_nominal,
            epochs=10)#batch_size=1024)
    #vaes[i][0].load_weights('Models/Weights/Nominal_weights_vae.hdf5')


FITTING Vae with beta = 1.0


Epoch 1/10


 32/300 [==>...........................] - ETA: 30s - loss: 1053.8440

 64/300 [=====>........................] - ETA: 13s - loss: 1102.5256

 96/300 [========>.....................] - ETA: 8s - loss: 1094.5135 

...

 96/300 [========>.....................] - ETA: 0s - loss: 1023.4152

128/300 [===========>..................] - ETA: 0s - loss: 1022.4033

192/300 [==================>...........] - ETA: 0s - loss: 1025.7526

256/300 [========================>.....] - ETA: 0s - loss: 1024.6094

300/300 [==============================] - 0s 2ms/step - loss: 1024.1426


In [55]:
def return_mask(num, labels):
    return np.squeeze(np.argwhere(labels == num))

labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_labels.csv")
labels = labels.drop(columns="Unnamed: 0")
labels = np.array(labels)

masks = [return_mask(num,labels)[:,0] for num in range(0,9)]


from sklearn.decomposition import PCA

model_index = 0
encodings = []

for i in vaes:
    encodings.append(i[1].predict(trainX_nominal))

#encodings = [encoder.predict(trainX_nominal)]

def check_z_sampling(encoded_values):
    m = encoded_values[0]
    var = np.exp(0.5*encoded_values[1])
    eps = np.random.normal(0,1,latent_dim)
    
    sampled = []
    index = 0
    var_zero = np.zeros(10)
    for means in m:
        sample = means+var[index]*eps
        #sample = means+var_zero*eps 
        sampled.append(sample)
        index += 1
    
    sampled = np.array(sampled)
    return sampled

def plot_pca(title, type): 
    x_val = []
    y_val= []
    for i in range(principalComponents.shape[0]):
        x_val.append(principalComponents[i][0])
        y_val.append(principalComponents[i][1])
    x_val = np.array(x_val)
    y_val = np.array(y_val)
    
    for mask in masks:
        plt.scatter(x=x_val[mask], y=y_val[mask], alpha=0.5)

    plt.legend(labels=np.arange(0,9))
    plt.title(str(title)+""+type)
    plt.show()
    
titles=['z_mean','z_log_var','z']
for i, encod in enumerate(encodings):
        for j in range(3):
            latent_values = check_z_sampling(encod)
            scaler = StandardScaler()
            enc_input = scaler.fit_transform(encod[j]) 
            pca = PCA(2)
            principalComponents = pca.fit_transform(enc_input)
            #print(pca.explained_variance_ratio_)
            plot_pca(betas[i], titles[j])


In [158]:
boat_strange = pd.read_csv("Data/Boat_nominal_data/Boat_unseen.csv")
boat_strange = boat_strange.drop(columns=["Unnamed: 0", "M0C", "M1C", 
                                  "Acceleration","Speed"])
scaler = MinMaxScaler()
boat_unseen_data = scaler.fit_transform(boat_strange)
#boat_unseen_data = np.array(boat_strange)
boat_unseen_data = np.reshape(boat_unseen_data, (1,len(boat_unseen_data),n_features))

In [159]:
unseen_encoding = vaes[0][1].predict(boat_unseen_data)
pca.transform(unseen_encoding[0])




array([[-1.7921036, -3.8114154]], dtype=float32)

In [57]:
runs = []
for mask in masks:
    run_for_class = trainX_nominal[mask]
    print(run_for_class.shape)
    runs.append(run_for_class)
    
for i in runs[0]:
    run = np.reshape(i, (1, 1024,4))
    rec = vaes[0][0].predict(run)
    rec = np.reshape(rec, (len(trainX_nominal[0]), n_features))
    reconstruction_df = pd.DataFrame(rec, columns=boat_csv.columns)
    
    plt.plot(reconstruction_df["Lon"], reconstruction_df["Lat"])
    #plt.savefig("Imgs/Latent_reconstruction/"+str(title)+".png")
    plt.show()
    break
    

(192, 1024, 4)
(15, 1024, 4)
(11, 1024, 4)
(12, 1024, 4)
(14, 1024, 4)
(11, 1024, 4)
(13, 1024, 4)
(16, 1024, 4)
(16, 1024, 4)


In [10]:

encodings = []

for i in vaes:
    encodings.append(i[1].predict(trainX_nominal))

def check_z_sampling(encoded_values):
    m = encoded_values[0]
    var = np.exp(0.5*encoded_values[1])
    eps = np.random.normal(0,1,latent_dim)
    
    sampled = []
    index = 0
    var_zero = np.zeros(10)
    for means in m:
        sample = means+var[index]*eps
        #sample = means+var_zero*eps 
        
        sampled.append(sample)
        index +=1
    
    sampled = np.array(sampled)
    return sampled
    
values = check_z_sampling(encodings[0])

labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_labels.csv")
labels = labels.drop(columns="Unnamed: 0")
labels = np.array(labels)

mat_mask = np.array([labels for i in range(latent_dim)])

from sklearn.decomposition import PCA

labels = np.array(pd.read_csv("Data/Boat_nominal_data/Boat_mix_labels.csv")['label']) 
nominals = np.squeeze(np.argwhere(labels==1))
anomalous = np.squeeze(np.argwhere(labels==0))
print(type(nominals), type(nominals[0]))

model_index = 0

titles = ["Mean", "Std", "Sampled"]

for i in encodings[model_index]:
    scaler = StandardScaler()
    enc_input = scaler.fit_transform(i)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(enc_input)
    x_val = []
    y_val=[]
    for i in range(principalComponents.shape[0]):
        x_val.append(principalComponents[i][0])
        y_val.append(principalComponents[i][1])
    x_val = np.array(x_val)
    y_val = np.array(y_val)
    
    
    plt.scatter(x=x_val[nominals],y=y_val[nominals], alpha=0.5)
    plt.scatter(x=x_val[anomalous],y=y_val[anomalous], alpha=0.5)
    plt.show()


from sklearn.manifold import TSNE

def tsne(data, title):
    tsne = TSNE(n_components=2, random_state=0)
    
    tsne_obj= tsne.fit_transform(data)
    tsne_df = pd.DataFrame({'X':tsne_obj[:,0],
                            'Y':tsne_obj[:,1],
                            })
    
    plt.scatter(x=tsne_df["X"][nominals],
                y=tsne_df["Y"][nominals], alpha=0.5)
    plt.scatter(x=tsne_df["X"][anomalous],
                y=tsne_df["Y"][anomalous], alpha=0.5)
    plt.title(title)
    plt.show()
    
    return tsne_df

for i, encode in enumerate(encodings[model_index]):
    tsne_enc_nom_df = tsne(encode, titles[i])

<class 'numpy.ndarray'> <class 'numpy.int64'>


In [62]:

def sample_from_latent_space(xCoord, yCoord):
    point = np.array([xCoord,yCoord])
    latent_point = pca.inverse_transform(point)
    return np.reshape(latent_point,(1, latent_dim))

def visualize_reconstruction(reconstructed_run, title):
    plt.plot(reconstructed_run["Lon"], reconstructed_run["Lat"])
    plt.title(title)
    
    # plt.plot(reconstructed_run["Sin"])
    # plt.plot(reconstructed_run["Cosin"])
    # plt.title(title)
    plt.savefig("Imgs/Latent_analysis/"+str(title)+".png")
    plt.clf()
    # plt.show()

X,Y = np.mgrid[-5:5:0.5, -5:5:0.5]
XY = np.vstack((X.flatten(), Y.flatten())).T
print(XY.shape)

for i,p in enumerate(XY):    
    point = sample_from_latent_space(p[0],p[1])
    reconstructed = np.reshape(vaes[model_index][2].predict(point), (len(trainX_nominal[0]),n_features))
    visualize_reconstruction(pd.DataFrame(reconstructed, columns=boat_csv.columns), 
                             title=str(i)+str(p))

# line = np.arange(-10, 10, 0.5)
# 
# for i,point2d in enumerate(line):
#     point = sample_from_latent_space(point2d,point2d)
#     reconstructed = np.reshape(vaes[model_index][2].predict(point), (len(trainX_nominal[0]),n_features))
#     visualize_reconstruction(pd.DataFrame(reconstructed, columns=boat_csv.columns), 
#                              title=point2d)    



(400, 2)


In [ ]:

def get_neuron_values(encoding):
    neurons_m = []
    neurons_var = []
    for i in range(latent_dim):
        neurons_m.append(encoding[0][:, i])
        neurons_var.append(encoding[1][:, i])
    
    for i in range(latent_dim):
        plt.plot(neurons_m[i])
    plt.show()
    
    return (neurons_m, neurons_var)

neuron_values = []
for i in encodings:
 neuron_values.append(get_neuron_values(i))


(10, 500, 1)


In [68]:

def nominal_parameters(n_m, n_var):
    neuron_avg_nom = np.ma.array(n_m, mask=np.logical_not(mat_mask))
    neuron_avg_nom = np.mean(neuron_avg_nom, axis=1)
    neuron_var_nom = np.ma.array(n_var, mask=np.logical_not(mat_mask))
    neuron_var_nom = np.mean(neuron_var_nom, axis=1)
     
    neuron_avg_anom = np.ma.array(n_m, mask=mat_mask)
    neuron_avg_anom = np.mean(neuron_avg_anom, axis=1)
    neuron_var_anom = np.ma.array(n_var, mask=mat_mask)
    neuron_var_anom = np.mean(neuron_var_anom, axis=1)
    
    return (neuron_avg_nom, neuron_var_nom, neuron_avg_anom, neuron_var_anom)

enc_values = []
for i in neuron_values:
    enc_values.append(nominal_parameters(i[0], i[1]))
print(len(enc_values))

10


In [69]:
def visualize_difference(avg_n, var_n, avg_an, var_an):
    plt.plot(avg_n)
    plt.plot(avg_an)
    plt.title("MEAN")
    plt.show()
    
    # plt.plot(var_n)
    # plt.plot(var_an)
    # plt.title("STD")
    # plt.show()


for i in enc_values:
    visualize_difference(i[0], i[1], i[2], i[3])


In [8]:
boat_anom_csv = pd.read_csv("Data/Boat_anom_sequence.csv")
boat_anom_csv = boat_anom_csv.drop(columns=["Unnamed: 0","Speed", "Acceleration",
                                            "M0C", "M1C"])
anom_data_norm = scaler.fit_transform(boat_anom_csv)
anom_data_norm = np.reshape(anom_data_norm, (1, len(anom_data_norm), n_features))
anom_data_encoding = np.squeeze(encoder.predict(anom_data_norm))

anom_means = anom_data_encoding[0]
anom_var = anom_data_encoding[1]
print(anom_means, anom_var)


[ 0.08811088  0.1255463   0.01701382 -0.08566583  0.1815099  -0.00088965] [ 0.16391774  0.00667964 -0.08026419 -0.01931977 -0.09087465  0.04685581]


In [130]:
def visualize_latent_sampled(sample_nom, sample_anom):
    plt.plot(sample_nom)    
    plt.plot(sample_anom)
    plt.title("SAMPLED DIFFERENCES")
    plt.show()


def reconstruct_from_latent_vectors(latents, model_index):
    latents = np.reshape(latents, (1,len(latents)))
    run = vaes[model_index][2].predict(latents)
    run_df = pd.DataFrame(run[0], columns=boat_csv.columns)
    
    plt.plot(run_df["Lon"])#, run_df["Lat"])
    plt.plot(run_df["Lat"])
    plt.show()
    

epsilon = np.random.normal(0,1)
sample_nominal = enc_values[0][0] + np.exp(0.5*enc_values[0][1])*epsilon
sample_anomalous = enc_values[0][2] + np.exp(0.5*enc_values[0][3])*epsilon





In [131]:
changes = np.linspace(0,6,50)
neur = 0
for i in changes:
    sample_nominal[neur] = sample_anomalous[neur]*i
    #visualize_latent_sampled(sample_nominal, sample_anomalous)
    reconstruct_from_latent_vectors(sample_nominal,0)
# reconstruct_from_latent_vectors(sample_anomalous)


In [30]:

from sklearn.decomposition import PCA

labels = np.array(pd.read_csv("Data/Boat_nominal_data/Boat_mix_labels.csv")['label']) 
nominals = np.squeeze(np.argwhere(labels==1))
anomalous = np.squeeze(np.argwhere(labels==0))
print(type(nominals), type(nominals[0]))

model_index = 0

titles = ["Mean", "Std", "Sampled"]

for i in encodings[model_index]:
    scaler = StandardScaler()
    enc_input = scaler.fit_transform(i)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(enc_input)
    x_val = []
    y_val=[]
    for i in range(principalComponents.shape[0]):
        x_val.append(principalComponents[i][0])
        y_val.append(principalComponents[i][1])
    x_val = np.array(x_val)
    y_val = np.array(y_val)
    
    
    plt.scatter(x=x_val[nominals],y=y_val[nominals], alpha=0.5)
    plt.scatter(x=x_val[anomalous],y=y_val[anomalous], alpha=0.5)
    plt.show()


from sklearn.manifold import TSNE

def tsne(data, title):
    tsne = TSNE(n_components=2, random_state=0)
    
    tsne_obj= tsne.fit_transform(data)
    tsne_df = pd.DataFrame({'X':tsne_obj[:,0],
                            'Y':tsne_obj[:,1],
                            })
    
    plt.scatter(x=tsne_df["X"][nominals],
                y=tsne_df["Y"][nominals], alpha=0.5)
    plt.scatter(x=tsne_df["X"][anomalous],
                y=tsne_df["Y"][anomalous], alpha=0.5)
    plt.title(title)
    plt.show()
    
    return tsne_df

for i, encode in enumerate(encodings[model_index]):
    tsne_enc_nom_df = tsne(encode, titles[i])


<class 'numpy.ndarray'> <class 'numpy.int64'>


In [33]:
def sample_from_latent_space(xCoord, yCoord):
    point = np.array([xCoord,yCoord])
    latent_point = pca.inverse_transform(point)
    return np.reshape(latent_point,(1, latent_dim))

def visualize_reconstruction(reconstructed_run, title):
    plt.plot(reconstructed_run["Lon"], reconstructed_run["Lat"])
    plt.title(title)
    plt.show()
    # plt.plot(reconstructed_run["Sin"])
    # plt.plot(reconstructed_run["Cosin"])
    # plt.show()

line = np.arange(-6, 6, 0.5)
X,Y = np.mgrid[-4:4:0.5, -4:4:0.5]
XY = np.vstack((X.flatten(), Y.flatten())).T
print(XY.shape)

for p in XY:    
    point = sample_from_latent_space(p[1],p[0])
    reconstructed = np.reshape(vaes[model_index][2].predict(point), (len(trainX_nominal[0]),n_features))
    visualize_reconstruction(pd.DataFrame(reconstructed, columns=boat_csv.columns), 
                             title=p)


(256, 2)


In [46]:
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering

plt.figure(figsize=(10, 7))
plt.title("Customer Dendograms")
dend_nom = shc.dendrogram(shc.linkage(tsne_enc_nom_df, method='ward'))


In [83]:
n_clusters = 2
cluster = AgglomerativeClustering(n_clusters=n_clusters, 
                                  affinity='euclidean',
                                  linkage='ward')
cl_nom = cluster.fit_predict(tsne_enc_nom_df)
print(cl_nom)

print(labels)


In [87]:
x_val_nom = []
y_val_nom = []

x_val_anom = []
y_val_anom = []
for i, xCoord in enumerate(tsne_enc_nom_df['X']):
    if cl_nom[i] == 0:
        x_val_nom.append(xCoord)
        y_val_nom.append(tsne_enc_nom_df['Y'][i])
    else:
        x_val_anom.append(xCoord)
        y_val_anom.append(tsne_enc_nom_df['Y'][i])

plt.scatter(x=x_val_nom,
            y=y_val_nom, alpha=0.5)
plt.scatter(x=x_val_anom,
            y=y_val_anom, alpha=0.5)
plt.show()
        
    



